libraries, constants and imports required for the program:
<ul>
<li>Import the enum class created by me</li>
<li>Define the file name</li>
<li>Define if using the default values or allows the user to insert them</li>
<li>Define the default location value</li>
<li>Define the default number of elements value</li>
</ul>

In [149]:
from Enum_data import EnumData

FILE_INPUT = "dati.csv"
ALLOW_USER_INPUT = 0
CITY = "Rome"
N = 5

This function is used to fill data missing as described by the rule

In [144]:
def fill_values(data:list[list], start:int, stop:int, first:float, last:float, col:int)->None:            
    while start < stop:
        data[start][col] = round((first+last)/2,3)
        first = data[start][col]
        start+=1

This function reads the file, drops the first line and fills the missing values: <br>
it creates the two enums required (one for temperature, one for missing values) <br>
if a measurement is missing the data are read until a the next usable value is found <br>


In [145]:
def read_file(FILE_INPUT:str) -> list[list[str, float, float, str, str, str, str]]:
    data = []

    with open(FILE_INPUT, "r", encoding='UTF-8') as fp:
        fp.readline()

        enumData = EnumData()
        missingTemp=-1
        missingDelta=-1


        for i, line in enumerate(fp):
            line = line.strip().split(",")
            try:
                line[enumData.AVG_TEMP] = round(float(line[enumData.AVG_TEMP]), 3)
            except ValueError:
                missingTemp = i if missingTemp == -1 else missingTemp

            try:
                line[enumData.AVG_DELTA] = round(float(line[enumData.AVG_DELTA]),3)
            except ValueError:
                missingDelta = i if missingDelta == -1 else missingDelta

            data.append(line)

            if isinstance(line[enumData.AVG_TEMP], float) and missingTemp != -1:
                fill_values(data, missingTemp, i,
                        data[missingTemp - 1][enumData.AVG_TEMP]
                        if missingTemp > 1
                        else 0,
                    line[enumData.AVG_TEMP], enumData.AVG_TEMP) 
                missingTemp = -1


            if isinstance(line[enumData.AVG_DELTA], float) and missingDelta != -1:
                fill_values(data, missingDelta, i,
                        data[missingDelta - 1][enumData.AVG_DELTA]
                        if missingDelta > 1
                        else 0,
                    line[enumData.AVG_DELTA], enumData.AVG_DELTA)
                missingDelta = -1

        if missingTemp != -1:
            fill_values(data, missingTemp, len(data), data[missingTemp - 1][enumData.AVG_TEMP],0, enumData.AVG_TEMP) 
            
        if missingDelta != -1:
            fill_values(data, missingDelta, len(data), data[missingDelta - 1][enumData.AVG_DELTA],0, enumData.AVG_DELTA) 
            

    return data

This function allows the user to input the name of the city and the number of rows wanted

In [146]:
def user_inputs(max:int)->tuple[str, int]:
    n = -1
    city = ""  
    
    while not city:
        city = input("Insert the city to look for : ").strip()
    
    while n < 1 or n>=max:
        try:
            n = int(input("Insert the number of rows : ").strip())
        except ValueError:
            n = -1
        
    return (city, n)
    

This function prints the N coldest and N hottest measurements for the given city

In [147]:
def searchNHottestColdest(data:list[list], info:list[str, int])->None:
    enum = EnumData()
    lista = sorted(filter(lambda x: x[enum.CITY]==info[0], data), key=lambda x:x[enum.AVG_TEMP])
    
    print(f"{info[1]} COLDEST MEASUREMENT FOR {info[0].upper()}")
    for item in lista[:info[1]]:
        print(f"Date = {item[enum.DATE]} \t Temperature ={item[enum.AVG_TEMP]}")
        
    print(f"\n\n{info[1]} HOTTEST MEASUREMENT FOR {info[0].upper()}")
    for item in lista[len(lista)-info[1]:]:
        print(f"Date = {item[enum.DATE]} \t Temperature ={item[enum.AVG_TEMP]}")

The main function that regulates the code execution:
<ol>
<li>Reads the input data</li>
<li>Checks which values should be used and allows the user to input them</li>
<li>Prints the N coldest/hottest measurements of the previously defined city</li>
<li>Prints the graph that shows the temperature land temperature for the city of Rome and Bangkok </li>
</ol>

In [148]:
def main()->None:
    data = read_file(FILE_INPUT)
    
    searchNHottestColdest(data, user_inputs(len(data)) if ALLOW_USER_INPUT else (CITY, N))
    
    
    
main()

5 COLDEST MEASUREMENT FOR Rome
Date = 1942-01-01 	 Temperature =-1.441
Date = 1956-02-01 	 Temperature =-1.304
Date = 1864-01-01 	 Temperature =-1.019
Date = 1830-01-01 	 Temperature =-0.871
Date = 1893-01-01 	 Temperature =-0.783


5 HOTTEST MEASUREMENT FOR Rome
Date = 2012-07-01 	 Temperature =24.48
Date = 2012-08-01 	 Temperature =24.731
Date = 1994-08-01 	 Temperature =24.873
Date = 2003-07-01 	 Temperature =24.998
Date = 2003-08-01 	 Temperature =25.951
